In [ ]:
#Copyright (c) 2025, Oracle and/or its affiliates.
#Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
pip install boto3

In [ ]:
import oci
import io
import zipfile
import time

# OCI Configuration
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('~/.oci/config', CONFIG_PROFILE)
object_storage = oci.object_storage.ObjectStorageClient(config)

In [ ]:
prefixes = [
    "fscmtopmodelam_finextractam_apbiccextractam",
    "fscmtopmodelam_finextractam_arbiccextractam",
    "fscmtopmodelam_finextractam_cebiccextractam",
    "fscmtopmodelam_finextractam_exmbiccextractam",
    "fscmtopmodelam_finextractam_fabiccextractam",
    "fscmtopmodelam_finextractam_funbiccextractam",
    "fscmtopmodelam_finextractam_glbiccextractam",
    "fscmtopmodelam_finextractam_iexbiccextractam",
    "fscmtopmodelam_finextractam_xlabiccextractam",
    "fscmtopmodelam_finextractam_xlebiccextractam",
    "fscmtopmodelam_finextractam_zxbiccextractam"
]

In [ ]:
import base64
def get_text_secret(secret_ocid):
    vault_client = oci.secrets.SecretsClient(config)
    try:
        secret_content = vault_client.get_secret_bundle(secret_ocid).data.secret_bundle_content.content
        decrypted_secret_content = base64.b64decode(secret_content).decode("utf-8")
    except Exception as ex:
        print("ERROR: failed to retrieve the secret content", ex, flush=True)
        raise
    return decrypted_secret_content

In [ ]:
import snowflake.connector
passowrd=get_text_secret("<oci vault secret>")

conn = snowflake.connector.connect(
    user='<>',
    password=passowrd,
    account='<>',
    warehouse='<>',
    database='<>',
    schema='<>'
)

cur = conn.cursor()

In [ ]:
import boto3
s3 = boto3.resource(
    's3',
    aws_access_key_id="<>",
    aws_secret_access_key="<>=",
    region_name="<region>", # Region name here that matches the endpoint
    endpoint_url="https://<OCI namespace>.compat.objectstorage.<region>.oraclecloud.com" # Include your namespace in the URL
)

In [ ]:
def uploadToS3(file_contents, destination_bucket_name, file_name):
    with zipfile.ZipFile(io.BytesIO(file_contents)) as zip_ref:
    # List contents of the zip file
        for file_info in zip_ref.infolist():
            extracted_file_content = zip_ref.read(file_info.filename)
            # Upload extracted file to destination bucket
            client = boto3.client('s3', region_name='<S3 region>')
            client.put_object(Bucket=destination_bucket_name,Key=process_filename(file_info.filename),
                              Body=io.BytesIO(extracted_file_content)
                             )

In [ ]:
def checkLisOfObjects(path_file):
    for substring in prefixes:
        if substring in path_file:
            return True

In [ ]:
def process_filename(filename):
    # Split the filename based on underscores
    parts = filename.split('_')
    
    # Ensure there are at least four underscores in the filename
    if len(parts) < 5:
        return "Invalid filename"
    
    # Join the parts after the fourth underscore
    new_filename = '_'.join(parts[4:])
    
    # Remove everything after the first hyphen
    new_filename = new_filename.split('-')[0]
    
    return new_filename+".csv"

In [ ]:
source_bucket = '<OCI Bucket>'
source_namespace='<OCI Namespace>'
next_starts_with = None
response = object_storage.list_objects(source_namespace, source_bucket,start=next_starts_with, fields='size,timeCreated,timeModified,storageTier', retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY)
next_starts_with= response.data.next_start_with
try:    
    start_time = time.time()
    print(start_time)
    for object_file in response.data.objects:
            path_file=str(object_file.name)
            if checkLisOfObjects(path_file):
                get_object_response = object_storage.get_object(source_namespace, source_bucket, path_file)
                uploadToS3(get_object_response.data.content, "<S3 Bucket>", path_file)
    if not next_starts_with:
        end_time = time.time()
        elapsed_time= end_time - start_time
        print("Elapsed time: ", elapsed_time)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
import boto3
s3 = boto3.resource('s3')
bucket = s3.Bucket('<S3 Bucket>')
start_time = time.time()
for obj in bucket.objects.all():
    object_name = obj.key[:-4]
    try:
        copy_table = f"COPY INTO {object_name} FROM '@<S3 Stage>/{object_name}.csv' FILE_FORMAT = (TYPE = 'csv', FIELD_DELIMITER=',', FIELD_OPTIONALLY_ENCLOSED_BY='\"', SKIP_HEADER = 1, error_on_column_count_mismatch=false)"
    except:
        copy_table = f"COPY INTO {object_name} FROM '@<S3 Stage>/{object_name}.csv' FILE_FORMAT = (TYPE = 'csv', FIELD_DELIMITER='|', FIELD_OPTIONALLY_ENCLOSED_BY='\"', SKIP_HEADER = 1,error_on_column_count_mismatch=false)"
    cur.execute(copy_table)
    conn.commit()
    print(copy_table)
end_time = time.time()
elapsed_time= end_time - start_time
print("Elapsed time: ", elapsed_time)
cur.close()
conn.close()